In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

Read the source file

In [3]:
uk_biobank = pd.read_csv("./resources//validation.csv")
uk_biobank.reset_index(drop=False,names=["id"], inplace=True)

/var/folders/z5/mj_15ff10rz3lfht0j5c9r680000gn/T/ipykernel_64831/3110670230.py:1: DtypeWarning: Columns (326,333,334,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712

## 1. Construct baseline predictors

we only need select the initial test result, since only initial age is available(21022-0.0)

select the column we want, that include all the biomarkers, date of birth and sex

In [4]:
# -0.0 means the first assesment of this field
col = {
    "eid": "eid",
    "31-0.0": "sex",
    "21022-0.0": "baseline_age",
    '30870-0.0': "triglycerides",
    '30780-0.0': "LDL",
    '30760-0.0': "HDL",
    '30750-0.0': "HbA1c",
    '30740-0.0': "glucose",
    '30700-0.0': "creatinine",
    '30871-0.0': "triglycerides_assay_date",
    '30781-0.0': "LDL_assay_date",
    '30761-0.0': "HDL_assay_date",
    '30751-0.0': "HbA1c_assay_date",
    '30741-0.0': "glucose_assay_date",
    '30701-0.0': "creatinine_assay_date",
}


we only select the patient the initial assessment result

In [5]:
predictors = filter(lambda x : x in list(col.keys()), uk_biobank.columns.to_list())
predictors = list(predictors)

In [6]:
data = uk_biobank[predictors]
data.rename(columns=col, inplace=True)

/var/folders/z5/mj_15ff10rz3lfht0j5c9r680000gn/T/ipykernel_64831/174750809.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns=col, inplace=True)


we want to filter the prediabetes patients, based on HbA1c and glucose

In [7]:
class HbA1c:
    @staticmethod
    def diagnose(value):
        if value < 39:
            return 0
        if value > 46:
            return 2
        else:
            return 1
        
class Glucose:
    @staticmethod
    def diagnose(value):
        if value < 5.6:
            return 0
        if value > 6.9:
            return 2
        else:
            return 1
        

In [8]:
hba1c = data["HbA1c"].apply(lambda x : HbA1c.diagnose(x) == 1)
glu = data["glucose"].apply(lambda x : Glucose.diagnose(x) == 1)
#union of two
pre = (hba1c + glu) > 0
data["hba1c"] = hba1c
data["glu"] = glu

prediabetes = data[pre]
# drop null value
prediabetes = prediabetes.dropna(how="any")

/var/folders/z5/mj_15ff10rz3lfht0j5c9r680000gn/T/ipykernel_64831/3304949585.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["hba1c"] = hba1c
/var/folders/z5/mj_15ff10rz3lfht0j5c9r680000gn/T/ipykernel_64831/3304949585.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["glu"] = glu


In [9]:
dates = [
    "triglycerides_assay_date",
    "LDL_assay_date",
    "HbA1c_assay_date",
    "glucose_assay_date",
    "creatinine_assay_date"
    ]

In [10]:
for d in dates:
    prediabetes[d] = pd.to_datetime(prediabetes[d])

In [11]:
def validate(series: pd.Series):
    for d in dates:
        test_date = series[d]
        b_date = series["baseline_date"]
        if test_date > b_date:
            return False
        if (b_date - test_date).days / 30.5 > 6:
            return False
        return True

In [12]:
v = []
baseline_date = []
for i in range(prediabetes.shape[0]):
    s = prediabetes.iloc[i]
    
    if s["hba1c"] and s["glu"]:
        baseline_date.append(min(s["glucose_assay_date"], s["HbA1c_assay_date"]))
    elif s["hba1c"]:
        baseline_date.append(prediabetes.iloc[i]["HbA1c_assay_date"])
    elif s["glu"]:
        baseline_date.append(prediabetes.iloc[i]["glucose_assay_date"])
    else:
        baseline_date.append(np.nan)
        
prediabetes["baseline_date"] = pd.to_datetime(baseline_date)

for i in range(prediabetes.shape[0]):
    v.append(validate(prediabetes.iloc[i]))

prediabetes["valid"] = v

valid cohort

In [13]:
cohort = prediabetes[v]

### Construct diagnosis table

In [15]:
diag_col = {
    "41280": "diagnosis_date",
    "41270": "diagnosis"
}

In [16]:
diag = filter(lambda x : x[:5] in list(diag_col.keys()), uk_biobank.columns.to_list())
diag = list(diag)

In [17]:
diag = list(filter(lambda x : x[:5] == '41270', uk_biobank.columns.to_list()))
diag_date = list(filter(lambda x : x[:5] == '41280', uk_biobank.columns.to_list()))

In [18]:
diag.extend(["eid"]) 
diag_date.extend(["eid"])

In [19]:
cohort_patient = cohort.eid

In [20]:
diag_date = uk_biobank[uk_biobank["eid"].isin(cohort_patient)][diag_date]
diag = uk_biobank[uk_biobank["eid"].isin(cohort_patient)][diag]

By the UK Bio-bank description, the latest date for the diagnosis is [2022-10-31](https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=41280), so we treat it as the truncation date.

In [21]:
# melt down the data frame
diag_date = diag_date.melt(id_vars=["eid"])
diag = diag.melt(id_vars=["eid"])

In [22]:
diag_date.dropna(how="any", inplace=True)
diag.dropna(how="any", inplace=True)

In [23]:
diag = diag[diag["value"].apply(lambda x : x[:3] == "E11")]
diag["visit"] = diag["variable"].apply(lambda x : x.split("-")[1])
diag_date["visit"] = diag_date["variable"].apply(lambda x : x.split("-")[1])

In [24]:
# join the table by the eid and visit index
diag = pd.merge(left=diag, right=diag_date[["eid", "visit", "value"]], on=["eid", "visit"])

In [25]:
# rename the column name
diag = diag.rename(columns={"value_x": "ICD10", "value_y": "diagnosis_date"})
# find the first record
diag = diag.sort_values("diagnosis_date").groupby("eid", as_index=False).apply(lambda x : x.head(1))

In [26]:
dataset = pd.merge(left=cohort[["eid", "creatinine", "glucose", "HbA1c", "HDL", "LDL", "triglycerides", "sex", "baseline_age", "baseline_date"]], right=diag, how="left", on="eid")
dataset["diagnosis_date"] = pd.to_datetime(dataset["diagnosis_date"])
dataset["progress_period"] = (dataset["diagnosis_date"] - dataset["baseline_date"]).apply(lambda x : x.days)

In [105]:
TIME_SPECTRUM = 2
TRUNCATION_DATE = "2022-10-31"

In [106]:
# # exclude the patients who is likely to have a incomplete record due to the truncation.
# ds = dataset[dataset["baseline_date"] + pd.Timedelta(days=TIME_SPECTRUM * 365.25) < TRUNCATION_DATE]
# filter out the patient whose diabetes diagnosis is earlier than baseline
ds = dataset.query(f"diagnosis_date.isnull() or progress_period > 0")

convert the unit of HbA1c since we use % in the model [reference](https://gpnotebook.com/simplepage.cfm?ID=x20121109220240605084)

In [107]:
from typing import List
import sys
sys.stdin = open("./resources/hba1c_convert.txt", "r")
s = sys.stdin.read()
sys.stdin.close()
splitted = s.split("\n")
percent = []
mmol = []
for line in splitted:
    percent.append(float(line.split("\t")[0]))
    mmol.append(float(line.split("\t")[1]))

def ceiling(nums: List[float], target: float):
    '''
    return the minimum number index in the nums that greater or equal to the target
    using binary search
    '''
    left = 0
    right = len(nums) - 1
    while(left < right):
        mid = (right - left) // 2 + left
        if nums[mid] == target:
            return mid
        if nums[mid] > target:
            right = mid - 1
        else:
            left = mid + 1
    return left

def linear_interpo(ymin: float, ymax: float, y: float, xmin: float, xmax: float):
    x: float = (y - ymin) * (xmax - xmin) / (ymax - ymin) + xmin
    return x

def convert(hba1c: float):
    head = ceiling(mmol, hba1c)
    if head == 0:
        return 4
    tail = head - 1
    return linear_interpo(mmol[tail], mmol[head], hba1c, percent[tail], percent[head])

In [108]:
ds["HbA1c"] = ds["HbA1c"].apply(convert)

/var/folders/z5/mj_15ff10rz3lfht0j5c9r680000gn/T/ipykernel_64831/2190840045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds["HbA1c"] = ds["HbA1c"].apply(convert)


In [109]:
t2dm_instance = ds.query(f"diagnosis_date.notnull() and progress_period < {TIME_SPECTRUM * 365.25}")
nont2dm_incidence = ds.query(f"diagnosis_date.isnull() or progress_period > {TIME_SPECTRUM * 365.25}")

t2dm_instance["class"] = 1
nont2dm_incidence["class"] = 0

ds = pd.concat([t2dm_instance, nont2dm_incidence])

/var/folders/z5/mj_15ff10rz3lfht0j5c9r680000gn/T/ipykernel_64831/1344338563.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2dm_instance["class"] = 1
/var/folders/z5/mj_15ff10rz3lfht0j5c9r680000gn/T/ipykernel_64831/1344338563.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nont2dm_incidence["class"] = 0


In [110]:
ds.shape

(31102, 16)

In [100]:
TIME_SPECTRUM

2

In [111]:
t2dm_instance["eid"].to_csv(f"year_{TIME_SPECTRUM}_t2dm.csv", index=False)
nont2dm_incidence["eid"].to_csv(f"year_{TIME_SPECTRUM}_nont2dm.csv", index=False)

In [112]:
X = ds[["creatinine", "glucose", "HbA1c", "HDL", "LDL", "triglycerides", "sex", "baseline_age"]]
y = ds[["class"]]

In [113]:
y.sum()

class    1581
dtype: int64

In [77]:
X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)

### Prediction

In [79]:
import os
import pickle
from sklearn import metrics
from dataclasses import dataclass
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

CHECK_POINT_PATH = r"../Output/A98_20230627_output/"
VALID_FEATURES = {
    2: ['creatinine', 'glucose', 'HbA1c', 'baseline_age'],
    5: [ 'HDL', 'creatinine', 'glucose','triglycerides','LDL','potassium_serum_plasma','HbA1c', 'baseline_age', 'sex'],
    10: ['creatinine', 'glucose', 'triglycerides', 'potassium_serum_plasma', 'HbA1c', 'baseline_age', 'sex']
}

model_path = os.path.join(CHECK_POINT_PATH, f"spec-{TIME_SPECTRUM}year", "models", "weighted_model")
scaler_path = os.path.join(CHECK_POINT_PATH, f"spec-{TIME_SPECTRUM}year", "models","scaler.pkl")
scaler = pickle.load(open(scaler_path, 'rb'))
model: tf.keras.Model = tf.keras.models.load_model(model_path)

/Users/george/miniforge3/envs/diab-logue/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RobustScaler from version 0.22.2.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [80]:
y.mean()

class    0.043663
dtype: float64

In [81]:
X_scaled = pd.DataFrame({})

In [82]:
VALID_TESTS = {
    2: ['HDL','choles','creatinine','glucose','triglycerides','LDL','potassium_serum_plasma','HbA1c'],
    5: ['HDL','choles','creatinine','glucose','triglycerides','LDL','potassium_serum_plasma','HbA1c'],
    10: ["choles", "creatinine", "glucose", "triglycerides", "potassium_serum_plasma", "HbA1c"]
}
for t in VALID_TESTS[TIME_SPECTRUM]:
    if t in X.columns:
        X_scaled[t] = X[t]
        assert X[t].isnull().sum() == 0
    else:
        X_scaled[t] = np.zeros(X.shape[0])


In [83]:
X_scaled.describe().T

,count,mean,std,min,25%,50%,75%,max
HDL,31102.0,1.457318,0.396370,0.347,1.172,1.398,1.68275,4.129
choles,31102.0,0.000000,0.000000,0.000,0.000,0.000,0.00000,0.000
creatinine,31102.0,72.374947,17.239708,26.600,61.700,70.700,81.00000,1086.900
glucose,31102.0,5.962253,0.382328,3.058,5.707,5.880,6.17600,12.142
triglycerides,31102.0,1.855400,1.067724,0.242,1.114,1.589,2.29800,11.227
LDL,31102.0,3.588003,0.871128,1.017,2.968,3.552,4.15500,9.610
potassium_serum_plasma,31102.0,0.000000,0.000000,0.000,0.000,0.000,0.00000,0.000
HbA1c,31102.0,5.430399,0.445381,4.000,5.230,5.420,5.54000,11.800


In [84]:
X_scaled = pd.DataFrame(scaler.transform(X_scaled), columns=VALID_TESTS[TIME_SPECTRUM])
X_scaled["baseline_age"] = X["baseline_age"]
X_scaled["sex"] = X["sex"]

/Users/george/miniforge3/envs/diab-logue/lib/python3.9/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RobustScaler was fitted without feature names
  warnings.warn(


In [85]:
X_scaled["potassium_serum_plasma"] = 0
X_scaled[VALID_FEATURES[TIME_SPECTRUM]].describe().T

,count,mean,std,min,25%,50%,75%,max
HDL,31102.0,0.245788,0.765194,-1.897683,-0.305019,0.131274,0.680985,5.403475
creatinine,31102.0,-0.165180,0.615704,-1.800000,-0.546429,-0.225000,0.142857,36.067857
glucose,31102.0,0.907196,0.523737,-3.071233,0.557534,0.794521,1.200000,9.372603
triglycerides,31102.0,0.826584,1.326366,-1.177640,-0.094410,0.495652,1.376398,12.468323
LDL,31102.0,0.438793,0.708119,-1.651114,-0.065193,0.409527,0.899691,5.333929
potassium_serum_plasma,31102.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
HbA1c,31102.0,-1.565338,1.484604,-6.333333,-2.233333,-1.600000,-1.200000,19.666667
baseline_age,31102.0,57.647064,7.871469,39.000000,52.000000,59.000000,64.000000,70.000000
sex,31102.0,0.475178,0.499392,0.000000,0.000000,0.000000,1.000000,1.000000


In [86]:
pred = model.predict(X_scaled[VALID_FEATURES[TIME_SPECTRUM]].to_numpy())

972/972 [==============================] - 0s 318us/step


In [87]:
print(f"time spectra:  {TIME_SPECTRUM}")
eva = model.evaluate(X_scaled[VALID_FEATURES[TIME_SPECTRUM]].to_numpy(), y.to_numpy())

time spectra:  5


972/972 [==============================] - 1s 413us/step - loss: 0.9583 - tp: 1148.0000 - fp: 17459.0000 - tn: 12285.0000 - fn: 210.0000 - accuracy: 0.4319 - precision: 0.0617 - recall: 0.8454 - auc: 0.7628 - prc: 0.2266


In [88]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y.to_numpy(), pred, pos_label=1)
metrics.auc(fpr, tpr)

0.7626943833327656